![](./img/sktime-logo-text-horizontal.jpg)

# Introduction to `sktime`

### Vision statement

- an easy-to-use, easy-to-extend, comprehensive **python framework** for ML and AI with time series
- **open source, permissive license, free to use**
- **openly and transparently governed**
- **friendly, responsive, kind and inclusive** community, with an active commitment to ensure fairness and equal opportunity
- an academically and commercially **neutral space**, with an **ecosystem integration** ambition and neutral point of view
- an **educational platform**, providing mentoring and upskilling opportunities for all career stages, especially early career


`sktime` is a vibrant, welcoming community with mentoring opportunities!

- We _love_ new contributors. Even if you are new to open source software development!
- Check out the `sktime` [new contributors guide](https://www.sktime.net/en/latest/get_involved/contributing.html)
- join our [discord](https://discord.com/invite/54ACzaFsn7) and/or one of our regular meetups!
- follow us on [LinkedIn](https://www.linkedin.com/company/scikit-time/)!


Further reading:

- `sktime` [notebook tutorials on binder](https://mybinder.org/v2/gh/sktime/sktime/main?filepath=examples)
- recorded [video tutorials](https://www.youtube.com/playlist?list=PLKs3UgGjlWHqNzu0LEOeLKvnjvvest2d0)
- find a bug or type? [tutorial feedback thread](https://github.com/sktime/sktime/issues/1447)

## Contents

`sktime` provides a unified, `scikit-learn`-like toolbox interface to multiple time series learning tasks.

**Section 1** explains what a `scikit-learn`-like toolbox is, using the example of `scikit-learn`.

**Section 2** gives an overview of learning with time series and challenges in the space.

**Section 3** gives a high-level engineering overview of `sktime`.

### 1. `sklearn` unified interface - the strategy pattern

`sktime` follows the `sklearn` / `skbase` interface:

- unified interface for objects/estimators
- modular design, strategy patterns
- composable, composites are interface homogeneous
- simple specification language and parameter interface
- visually informative pretty printing

`sklearn` provides a unified interface to multiple learning tasks including classification, regression.

any (supervised) estimator has the following interface points

1. **Instantiate** your model of choice, with parameter settings
2. **Fit** the instance of your model
3. Use that fitted instance to **predict** new data!

![](./img/estimator-conceptual-model.jpg)

the above in code:

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# get data to use the model on
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
from sklearn.svm import SVC

# 1. Instantiate SVC with parameters gamma, C
clf = SVC(gamma=0.001, C=100.0)

# 2. Fit clf to training data
clf.fit(X_train, y_train)

# 3. Predict labels on test data
y_test_pred = clf.predict(X_test)

y_test_pred

IMPORTANT: to use another classifier, only the specification line, part 1 changes!

`SVC` could have been `RandomForest`, steps 2 and 3 remain the same - unified interface:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Instantiate SVC with parameters gamma, C
clf = RandomForestClassifier(n_estimators=100)

# 2. Fit clf to training data
clf.fit(X_train, y_train)

# 3. Predict labels on test data
y_test_pred = clf.predict(X_test)

y_test_pred

In object oriented design terminology, this is called **"strategy pattern"**

= different estimators can be switched out without change to the interface

= like a power plug adapter, it's plug&play if it conforms with the interface

Pictorial summary:
![](./img/sklearn-unified-interface.jpg)

parameters can be accessed and set via `get_params`, `set_params`:

In [ ]:
clf.get_params()

### 2. `sktime` is devoted to time-series data analysis

Richer space of time series tasks, compared to "tabular":

- **Forecasting** - predict energy consumption tomorrow, based on past weeks
- **Classification** - classify electrocardiograms to healthy/sick, based on prior examples
- **Regression** - predict compound purity in bioreactor based on temperature/pressure profile
- **Clustering** - sort outlines of tree leaves into a small number of similar classes
- **Annotation** - identify jumps, anomalies, events in a data stream

`sktime` aims to provide `sklearn`-like, modular, composable, interfaces for these!

| Task | Status | Links |
|------|--------|-------|
| **Forecasting**     |  stable      |>>[Tutorial](https://www.sktime.net/en/latest/examples/01_forecasting.html) >> [API Reference](https://www.sktime.net/en/latest/api_reference/forecasting.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/forecasting.py)|
| **Time Series Classification** | stable |>>[Tutorial](https://github.com/sktime/sktime/blob/main/examples/02_classification.ipynb) >> [API Reference](https://www.sktime.net/en/latest/api_reference/classification.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/classification.py)|
| **Transformations** | stable |>>[Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) >> [API Reference](https://www.sktime.net/en/latest/api_reference/transformations.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)|
| **Parameter fitting** | maturing |>>[API Reference](https://www.sktime.net/en/latest/api_reference/param_est.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)|
| **Time Series Clustering** | maturing |>>[API Reference](https://www.sktime.net/en/latest/api_reference/clustering.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/clustering.py)|
| **Time Series Distances/Kernels** | maturing |>>[Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) >> [API Reference](https://www.sktime.net/en/latest/api_reference/dists_kernels.html) >> [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/dist_kern_panel.py)|
| **Annotation** | experimental |>>[Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/annotation.py)|
|**Distributions and simulation** | experimental |  None  |

**Some examples below - details in the respective module introduction notebooks!**

Example - forecasting

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.naive import NaiveForecaster

# step 1: data specification
y = load_airline()

# step 2: specifying forecasting horizon
fh = range(1, 37)

# step 3: specifying the forecasting algorithm
forecaster = NaiveForecaster(strategy="last", sp=12)

# step 4: fitting the forecaster
forecaster.fit(y)

# step 5: querying predictions
y_pred = forecaster.predict(fh)

In [ ]:
from sktime.utils.plotting import plot_series

fig, ax = plot_series(y, y_pred, labels=["train", "forecast"])

Example - classification

![](./img/tsc.png)

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_osuleaf
from sktime.dists_kernels import ScipyDist
from sktime.dists_kernels.compose_tab_to_panel import FlatDist

# step 1 - specify training data
X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")

# step 2 - specify data to predict labels for
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]

# step 3 - specify the classifier
mean_eucl_dist = FlatDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)

# step 4 - fitting the classifier
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_new.shape

In [ ]:
y_pred.shape

### 3. `sktime` integrates the time series modelling ecosystem!

the package space for time series is highly fragmented:

- lots of great implementations and methods out there!
- but many different interfaces, not composable like `sklearn`

`sktime` integrates the ecosystem - in friendly collaboration with all the packages out there!

- unified interface standard
- highly composable
- mini-package manager on estiator/module level
- easily extensible - 3rd party plugins, other packages

![](./img/unified_framework.png)

### 4. Summary

- `sklearn` interface: unified interface (strategy pattern), modular, composition stable, easy specification language
- `sktime` evolves the interface for time series learning tasks
- `sktime` integrates a fragmented ecosytem with interface, composability, dependency management

<hr>

### Credits: notebook 0 - `sktime` and `sklearn` intro

notebook creation: fkiraly, marrov

some vignettes based on existing `sktime` tutorials, credit: fkiraly, miraep8

slides (png/jpg):
* from fkiraly's postgraduate course at UCL, Principles and Patterns in Data Scientific Software Engineering
* ecosystem slide: fkiraly, mloning
* learning tasks: fkiraly, mloning

<hr>

#### many thanks to [all `sktime` contributors](https://www.sktime.net/en/latest/about/contributors.html)!

Citations & credits in academic research papers:

`sktime` toolbox: [sktime: A unified interface for machine learning with time series](https://arxiv.org/abs/1909.07872)

`sktime` design principles: [Designing machine learning toolboxes: Concepts, principles and patterns](https://arxiv.org/abs/2101.04938)